# Logging
> Production-ready logging capabilities for Cosma agents

In [ ]:
#|default_exp logging


## Overview

This module provides structured logging for agents running in production environments. It follows best practices for:
- JSON-structured logs for easy parsing
- Separate development and production handlers
- Event-based logging for analysis
- Token usage and performance metrics
- Container-friendly file handling



## Usage Example

```python
# Create an agent with production logging
agent = Agent(
    role="production_agent",
    model="gpt-4o",
    tools=[solve_math],
    log_dir="/var/log/cosma"  # Production log directory
)

# Logs will include structured data
response = agent.run_with_tools("Calculate sqrt(16)")
# Log output example:
# {"timestamp": "2024-02-15T14:23:45", 
#  "event": "tool_call", 
#  "data": {"tool": "solve_math", "args": {"expression": "sqrt(16)"}}}
```



## Production Setup

For containerized environments:
1. Mount a volume for logs: `/var/log/cosma`
2. Set LOG_LEVEL environment variable
3. Configure log rotation (logrotate recommended)


In [ ]:
# | export
from cosma.core import *
from fastcore.utils import *
from fastcore.basics import patch
import logging
import json
from pathlib import Path
from dataclasses import dataclass, field
from typing import Any, Optional, List, Callable, Dict


In [ ]:
# | export
@dataclass
class AgentLogger:
    """Structured logging for agent activities with console and file output.
    
    Args:
        name: Logger name (typically agent role)
        level: Logging level (default: INFO)
        log_dir: Optional directory for log files
        
    Example:
        ```python
        logger = AgentLogger(
            name="math_agent",
            log_dir="logs"
        )
        logger.log_event("tool_called", 
            tool="solve_math",
            input="sqrt(16)",
            result=4.0
        )
        ```
    """
    name: str
    level: int = logging.INFO
    log_dir: Optional[str] = None
    
    def __post_init__(self):
        """Setup console and optional file handlers with formatters."""
        self.logger = logging.getLogger(f"agent.{self.name}")
        self.logger.setLevel(self.level)
        
        # Prevent duplicate handlers
        if not self.logger.handlers:
            self._setup_console_handler()
            if self.log_dir: self._setup_file_handler()


In [ ]:
# | export
@patch
def _get_formatter(self:AgentLogger):
    """Create JSON-structured log formatter with metadata.
    
    Format: [timestamp] logger_name - level - {"event": event_type, "data": {event_data}}
    """
    return logging.Formatter(
        '[%(asctime)s] %(name)s - %(levelname)s - '
        '{"event": "%(event)s", "data": %(message)s}'
    )

@patch
def _setup_console_handler(self:AgentLogger):
    """Configure console output with structured formatting."""
    console = logging.StreamHandler()
    console.setFormatter(self._get_formatter())
    self.logger.addHandler(console)

@patch
def _setup_file_handler(self:AgentLogger):
    """Configure file output with structured formatting and rotation."""
    Path(self.log_dir).mkdir(parents=True, exist_ok=True)
    file_handler = logging.FileHandler(
        Path(self.log_dir)/f"{self.name}.log"
    )
    file_handler.setFormatter(self._get_formatter())
    self.logger.addHandler(file_handler)


In [ ]:
# | hide
# Test handler setup
test_logger = AgentLogger(
    name="test_agent",
    log_dir="test_logs"
)

# Verify handlers were created
print("Logger handlers:")
for h in test_logger.logger.handlers:
    print(f"- {type(h).__name__}")

# Verify log directory
if test_logger.log_dir:
    print(f"\nLog directory created: {Path(test_logger.log_dir).exists()}")


Logger handlers:
- StreamHandler
- FileHandler

Log directory created: True


In [ ]:
# | export
@patch
def log_event(self:AgentLogger, event:str, **data):
    """Log a structured event with arbitrary data.
    
    Args:
        event: Type of event (e.g., 'prompt_received', 'tool_called')
        **data: Arbitrary key-value pairs for event data
        
    Example:
        ```python
        logger.log_event('tool_called',
            tool_name='solve_math',
            input='sqrt(16)',
            result=4.0,
            execution_time=0.05
        )
        ```
    """
    extra = {'event': event}
    self.logger.info(json.dumps(data), extra=extra)


In [ ]:
# | hide
# Test event logging
test_logger = AgentLogger("test_agent", log_dir="test_logs")

print("Testing event logging:")
# Test prompt event
test_logger.log_event("prompt_received",
    prompt="What is 2+2?",
    timestamp="2024-02-15T14:30:00"
)

# Test tool usage event
test_logger.log_event("tool_called",
    tool="solve_math",
    input="2+2",
    result=4.0,
    execution_time=0.05
)

# Read back the last few lines from the log file
log_file = Path("test_logs")/f"test_agent.log"
print("\nLast few log entries:")
if log_file.exists():
    with open(log_file) as f:
        print('\n'.join(f.readlines()[-2:]))


[2025-02-15 11:11:22,663] agent.test_agent - INFO - {"event": "prompt_received", "data": {"prompt": "What is 2+2?", "timestamp": "2024-02-15T14:30:00"}}
[2025-02-15 11:11:22,664] agent.test_agent - INFO - {"event": "tool_called", "data": {"tool": "solve_math", "input": "2+2", "result": 4.0, "execution_time": 0.05}}


Testing event logging:

Last few log entries:
[2025-02-15 11:11:22,663] agent.test_agent - INFO - {"event": "prompt_received", "data": {"prompt": "What is 2+2?", "timestamp": "2024-02-15T14:30:00"}}

[2025-02-15 11:11:22,664] agent.test_agent - INFO - {"event": "tool_called", "data": {"tool": "solve_math", "input": "2+2", "result": 4.0, "execution_time": 0.05}}



In [ ]:
# | export
@dataclass
class AgentMetrics:
    """Collect and track agent performance metrics.
    
    Tracks:
    - Token usage
    - Tool calls
    - Response times
    - Success/failure rates
    """
    total_tokens: int = 0
    prompt_tokens: int = 0
    completion_tokens: int = 0
    tool_calls: int = 0
    total_time: float = 0.0
    successful_calls: int = 0
    failed_calls: int = 0

@patch
def log_metrics(self:AgentLogger, metrics:dict):
    """Log accumulated metrics for the agent.
    
    Args:
        metrics: Dictionary of metric names and values
    """
    self.log_event('metrics_update', **metrics)

@patch
def log_completion(self:AgentLogger, chat_completion, execution_time:float):
    """Log metrics from a chat completion response.
    
    Args:
        chat_completion: Response from cosette Chat
        execution_time: Time taken for the complete interaction
    """
    metrics = {
        'tokens': chat_completion.usage,
        'execution_time': execution_time,
        'model': chat_completion.model
    }
    self.log_metrics(metrics)


In [ ]:
# | hide
# Test metrics logging
import time
test_logger = AgentLogger("test_agent", log_dir="test_logs")

# Simulate a chat completion response
@dataclass
class MockCompletion:
    usage: dict = field(default_factory=lambda: {"total_tokens": 150, "prompt_tokens": 100, "completion_tokens": 50})
    model: str = "gpt-4o"

# Test metrics logging
start_time = time.time()
time.sleep(0.1)  # Simulate some work
completion = MockCompletion()
test_logger.log_completion(completion, time.time() - start_time)

# Show log output
log_file = Path("test_logs")/f"test_agent.log"
print("Last log entry:")
if log_file.exists():
    with open(log_file) as f:
        print(f.readlines()[-1])


[2025-02-15 11:13:02,569] agent.test_agent - INFO - {"event": "metrics_update", "data": {"tokens": {"total_tokens": 150, "prompt_tokens": 100, "completion_tokens": 50}, "execution_time": 0.10320520401000977, "model": "gpt-4o"}}


Last log entry:
[2025-02-15 11:13:02,569] agent.test_agent - INFO - {"event": "metrics_update", "data": {"tokens": {"total_tokens": 150, "prompt_tokens": 100, "completion_tokens": 50}, "execution_time": 0.10320520401000977, "model": "gpt-4o"}}

